In [ ]:
# Make sure to install necessary dependencies
! pip install chromadb
! pip install sentence_transformers 
! pip install --upgrade huggingface_hub==0.26.0
! pip install numpy , pandas

In [2]:
import pandas as pd
import os
import numpy as np
import chromadb
from sentence_transformers import SentenceTransformer
from chromadb import Client
import timeit
from clean_genres import clean_genres

c:\Users\pspra\anaconda3\envs\transformers\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class RecommendMovies:
    '''
    Class to recommend movies based on a given query.
    '''
    def __init__(self,movies_df_path,num_movies=5,num_rows=None):
        '''
        Initialize the RecommendMovies class.
        Parameters:
            movies_df_path (str): Path to the movies dataframe.
            num_movies (int): Number of movies to recommend. Deafult 5
            num_rows (int): (Dataset size) Number of rows to sample from the dataframe. Default None. Specifed by the user
            query_string (str): Query string to search for movies.
            model (SentenceTransformer): SentenceTransformer model to use for embeddings.
            client (chromadb.Client): ChromaDB client to use for database operations.
            collection (chromadb.Collection): ChromaDB collection to use for database operations.
            movie_embeddings (np.ndarray): Movie embeddings for the movies in the dataframe.
            query_embedding (np.ndarray): Query embedding for the query string.
        Returns:
            None
        '''
        assert movies_df_path is not None, f'movies_df_path should not be None'
        assert os.path.exists(movies_df_path), f'movies_df_path {movies_df_path} does not exist'

        assert num_movies is not None, f'num_movies should not be None'
        assert isinstance(num_movies, int), f'num_movies should be an integer, but got {type(num_movies)}'
        assert num_movies > 0, f'num_movies should be greater than 0, but got {num_movies}'

        self.movies_path = movies_df_path
        self.num_rows = num_rows
        self.random_state = 42
        self.movies_df = pd.read_csv(self.movies_path)
        if self.num_rows is not None:
            assert isinstance(self.num_rows, int), f'num_rows should be an integer, but got {type(self.num_rows)}'
            assert self.num_rows > 0, f'num_rows should be greater than 0, but got {self.num_rows}'
            assert self.num_rows <= len(self.movies_df), f'Number of rows {self.num_rows} is greater than the number of rows in the dataframe {len(self.movies_df)}'
            self.movies_df = self.movies_df.sample(n=self.num_rows, random_state=self.random_state)
            self.movies_df = self.movies_df.reset_index(drop=True)
        self.query_string = None
        self.num_movies = num_movies
        self.batch_size = 64

        self.processed_df = None
        self.valid_genres = set()
        self.model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
        self.movie_embeddings = None
        self.query_embedding = None
        
        
        print('Starting data preprocessing')
        preprocessing_time = timeit.timeit(self.preprocess_data, number=1)
        print(f'Data Preprocessing completed in {preprocessing_time:.4f} seconds')

        print(f'Starting database Engine')
        self.client = chromadb.Client()

        
        existing_collections = self.client.list_collections()
        if "movie_recommender" in existing_collections:
            print(f'Collection already exists')
            print(f'Loading existing collection')
            self.collection = self.client.get_collection(name="movie_recommender")
        else:
            print(f'Creating a new collection')
            self.collection = self.client.create_collection(name="movie_recommender")
            print('Starting Vector database creation')
            vector_data = timeit.timeit(self.create_embeddings, number=1)
            print(f'Vector database creation completed in {vector_data:.4f} seconds')

    
    def preprocess_data(self):
        '''
        The dataset used here is wikipedia movie plots dataset (https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots/data).
        Our columns of interest are Title, Genre and Plot.
        This function is used to preprocess the data. The preprocessing steps include:
        1. Selecting the columns of interest
        2. Dropping the rows with missing values
        3. Dropping the duplicate rows
        4. Cleaning the genres : The clean_genres function is used to clean the genres. The genres are cleaned by removing the unwanted characters and splitting the genres into a list of known genres.
        5. Creating a set of valid genres : The valid genres are created by splitting the genres into a list of known genres. The valid genres are used to filter the movies based on the genre.
        Parameters:
            None
        Returns:
            None
        '''
        self.processed_df = self.movies_df[['Title', 'Genre', 'Plot']]
        self.processed_df = self.processed_df.dropna()
        self.processed_df = self.processed_df.drop_duplicates()
        self.processed_df['Genre'] = clean_genres(self.processed_df)
        self.processed_df['Genre'].apply(lambda x: self.valid_genres.update(x.split('|')))
        
    
    def create_metadata(self,df_list):
        '''
        This function is used for filtering the entire database. To make the retrieval faster, we will create a metadata dictionary for each movie.
        The metadata dictionary will contain the genre of the movie as key value pairs. As a result we query only the movies that are in the same genre as the query.
        Parameters:
            df_list (list): List of genres for each movie obtained as a pipe-separated string from dataset
        Returns:
            (dict) : Dictionary with key value pairs indicating Genre of the movie
        '''
        metadata = []

        for i in range(len(df_list)):
            genres = df_list[i].split('|')
            metadata_dict = {'genre_combined': df_list[i]}  
        
            for genre in genres:
                metadata_dict[f'{genre.strip().lower()}'] = True
            
            metadata.append(metadata_dict)

        return metadata
    
    def create_embeddings(self):
        '''
        This is the function that is used for storing the movies into the vector database.
        The movie summaries are first encoded using a sentence transformer model to create embeddings that capture semantic meaning.
        The embeddings are then stored in the vector database along with the metadata and ids of the movies.
        The metadata is used for filtering the movies based on the genre.
        Parameters:
            None
        Returns:
            None
        '''
        self.movie_embeddings = self.model.encode(self.processed_df['Plot'].tolist())
        print(f'Generated Vector embeddings for {len(self.movie_embeddings)} movies')

        num_rows = len(self.processed_df)
        for start_idx in range(0, num_rows, self.batch_size):
            end_idx = min(start_idx + self.batch_size, num_rows)
            batch_documents = self.processed_df['Plot'].tolist()[start_idx:end_idx]
            batch_embeddings = self.movie_embeddings[start_idx:end_idx]
            batch_metadatas = self.create_metadata(self.processed_df['Genre'][start_idx:end_idx].tolist())
            batch_ids = self.processed_df.index.astype(str).tolist()[start_idx:end_idx]
            
            self.collection.upsert(
                documents=batch_documents,
                embeddings=batch_embeddings,
                metadatas=batch_metadatas,
                ids=batch_ids
            )
            # print(f'\tInserted batch of documents:',{start_idx}, 'to', {end_idx})

    
    def analyze_genre_from_query(self, query):
        '''
        This function is used to extract the genres that user has mentioned in the query.
        The genres are extracted by splitting the query into words and checking if the words are in the valid genres.

        Parameters:
            query (str): Query string to analyze for genres
        Returns:
            (list): List of genres from the query string compatible with chromadb query method for faster query
            raises ValueError if no valid genres are found in the query.
        '''
        words = query.strip().split(' ')
        # Convert to lowercase and remove duplicates
        words = set(word.lower() for word in words)
        extracted_genres = words & self.valid_genres
        if not extracted_genres:
            print(f'No valid genres found in the query. Please check the query and try again. Defaulting to "Comedy" genre')
            # raise ValueError(f'No valid genres found in the query. Please check the query and try again')
        
        # Create a query filter dictionary
        genre_filter = {
            'genre_combined': '|'.join(sorted(list(extracted_genres)))
        }
        
        # Add individual genre flags
        for genre in extracted_genres:
            genre_filter[f'{genre.strip().lower()}'] = True
        
        return genre_filter

    def delete_collection(self):
        '''
        This function is used to delete the collection from the database.
        The function first checks if the collection exists and then deletes the collection.
        Parameters:
            None
        Returns:
            None
        '''
        if self.client.get_collection(name="movie_recommender") is not None:
            self.client.delete_collection(name="movie_recommender")
            print('Collection deleted')
        else:
            print('Collection does not exist')

        
    def recommend_movies(self,query,num_movies=5):
        '''
        This function is used as an abstrcation to recommend movies based on the query provided by the user.
        The function first checks if the query is valid and then creates a query embedding using the sentence transformer model.
        The query embedding is then used to query the vector database for the most similar movies.
        The query is filtered based on the genres that are present in the query.
        Parameters:
            query (str): Query string to search for movies.
            num_movies (int): Number of movies to recommend. Default 5
        Returns:
            (pd.DataFrame): DataFrame containing the recommended movies and their scores.
        '''

        assert query is not None, f'query should not be None'
        assert isinstance(query, str), f'query should be a string, but got {type(query)}'
        assert num_movies is not None, f'num_movies should not be None'
        assert isinstance(num_movies, int), f'num_movies should be an integer, but got {type(num_movies)}'
        assert num_movies > 0, f'num_movies should be greater than 0, but got {num_movies}'

        self.num_movies = num_movies
        self.query_string = query
        self.query_embedding = self.model.encode([self.query_string])
        genre_filter = self.analyze_genre_from_query(self.query_string)
        
        or_conditions = [{'NA':True}]
        if genre_filter['genre_combined'] =='':
            or_conditions.append({'comedy':True})
        for key, value in genre_filter.items():
            if key != 'genre_combined' and value is True:
                or_conditions.append({key: True})
        
        if or_conditions:
            results = self.collection.query(
                query_embeddings=self.query_embedding,
                n_results=self.num_movies,
                where={"$or": or_conditions}
            )
        else:
            results = self.collection.query(
                query_embeddings=self.query_embedding,
                n_results=self.num_movies
            )
        
        ids = [int(i) for i in results['ids'][0]  ]
        returned_df = self.processed_df.loc[ids]
        returned_df['Score'] = results['distances'][0]
        returned_df.drop(columns=['Plot'],inplace=True)
        return returned_df

# Step 1
## Dataset Download
The Dataset [Link]('https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots') needs to be downloaded prior to running the script. Manual download is the quickest and easiest option. Otherwise downloading from kagglehub requires authentication which is not feasible if there is no kaggle account. \
Download and extract the csv file to the location where the script exists or make sure to have the location of the file which is needed in step 2

**Note**: If the link doesn't work copy paste the URL in the browser.

# Step 2
## Class Instantiation
Create an instance of the Movie recommender class \
Arguments :
1) **movies_df_path** - Path to the movies csv file. Must use the 
2) **num_rows (optional)** - This parameter specifies the no. of data points we want our vector database to hold. \
\
When the class is instantiated a vector database is created using the num of samples specified by the num_rows arguments by converting randomly sampled data points from the csv file. \
**Note**:
This process takes some time depending on the size of the data we want to store in our vector database. The larger data we want to process the more time it takes to convert the text information into vector database. But once done querying the database is extremely fast and robust. 

In [4]:
recommender = RecommendMovies(movies_df_path='wiki_movie_plots_deduped.csv',num_rows=1500)

Starting data preprocessing
Data Preprocessing completed in 0.1455 seconds
Starting database Engine
Creating a new collection
Starting Vector database creation
Generated Vector embeddings for 1500 movies
Vector database creation completed in 96.8145 seconds


# Step 3:
## Inference:
We use the recommend_movies function the class to obtain the set of recommended movies. \
Arguments:
1) **query** : A query string from the user to indicate which types of movies the user want. A typical query contains ceratin keyworsd related to the genre the user wants \
    **Ex: "I like to watch romantic movies with subtle comedy"**
2) **num_movies** : Number of movies the user wants us to recommend


In [5]:
recommender.recommend_movies(query='I want to watch action packed thriller movies', num_movies=20)

,Title,Genre,Score
25,Love Camp 7,thriller,1.395729
831,Chhalia,action,1.460976
632,Sex & Fury,action,1.476759
427,Hotel California,action|comedy,1.510511
292,8 Thottakkal,crime|thriller,1.520951
1170,A Dandy in Aspic,spy|thriller,1.539236
61,Hotel Chelsea,thriller,1.552596
467,Hiraasat,action|crime|drama,1.558935
1390,"War, Inc.",action,1.577744
1219,XX: Beautiful Prey,adult|thriller,1.578278


In [6]:
recommender.recommend_movies(query='Spy investigation', num_movies=15)

,Title,Genre,Score
1004,From Russia with Love,action|spy,1.129512
11,After Tonight,spy,1.321421
1394,Die Another Day,action|spy,1.337664
1170,A Dandy in Aspic,spy|thriller,1.348758
598,The Mole Song: Hong Kong Capriccio,action|comedy|drama|science_fiction|spy|superhero,1.365715
372,Licensed to Love and Kill,action|spy,1.400530
594,The Naked Runner,spy,1.553648
551,Fairy Tail: Dragon Cry,action|adventure|drama|fantasy|mystery|spy,1.777670


In [7]:
recommender.recommend_movies(query='I prefer war documentaries', num_movies=5)

,Title,Genre,Score
705,Dunkirk,war,1.290211
746,The Eagle and the Hawk,drama|war,1.366564
1414,San Demetrio London,war,1.443386
737,Desperate Journey,war,1.477732
1388,The Lighthorsemen,drama|war,1.489172


In [8]:
recommender.recommend_movies(query='comedy or rom-com or romance', num_movies=10)

,Title,Genre,Score
1296,"I Will, I Will... for Now",comedy,1.243847
447,Insomnia Lover,comedy|romance,1.315180
969,Lunch Hour,comedy,1.337848
917,Gas-s-s-s,comedy,1.443974
1399,Maindhan,action|comedy,1.468618
1471,Lock Up Your Daughters!,comedy,1.488819
1345,Valentine's Day,comedy|romance,1.503200
153,Her Minor Thing,comedy,1.504155
507,Happy Though Married,comedy,1.517400
722,How to Murder Your Wife,comedy,1.518565


In [9]:
recommender.recommend_movies(query='horror', num_movies=10)

,Title,Genre,Score
208,The Avenging Conscience,drama|horror,1.371714
875,Halloween II,horror,1.413355
245,Black Water Vampire,horror,1.448597
1460,Tower of Evil,horror,1.452735
1445,"Conjuring, TheThe Conjuring",horror|supernatural,1.452866
1207,Teenage Zombies,horror,1.476791
40,Vampire in Brooklyn,comedy|horror,1.487868
645,Aftershock,horror|thriller,1.494793
109,The Mirror,horror,1.507972
461,The Sender,horror,1.515146


In [10]:
recommender.recommend_movies(query='Animation', num_movies=10)

,Title,Genre,Score
1476,Tony de Peltrie,animation,1.268086
39,Who Killed Who?,animation|short,1.455070
647,The Magic Roundabout,animation,1.536697
480,Crayon Shin-chan: The Storm Called: Operation ...,animation,1.639123
197,That's My Mommy,animation,1.644740
789,Typhoon Noruda,animation,1.653150
761,Millennium Actress,animation|drama,1.681293
548,Sahara Hare,animation|short,1.685677
391,Hooked Bear,animation,1.691521
184,The Scarecrow,animation|fantasy,1.696110


In [11]:
recommender.recommend_movies(query='I watch movies', num_movies=10)

No valid genres found in the query. Please check the query and try again. Defaulting to "Comedy" genre


,Title,Genre,Score
1023,Angels in Stardust,comedy,1.487885
1391,An Alan Smithee Film: Burn Hollywood Burn,comedy,1.498703
1363,Simon,comedy,1.505658
922,Morons from Outer Space,comedy|science_fiction,1.517438
1399,Maindhan,action|comedy,1.528619
116,"Gaily, Gaily",comedy,1.556756
917,Gas-s-s-s,comedy,1.571054
1209,Pixels,animation|comedy,1.576107
994,The Spy Next Door,action|comedy|family,1.577899
427,Hotel California,action|comedy,1.578007


# Explanation
The choice of vector databases is not arbitrary. The same problem can be tackled with a simple cosine similarity based tf-idf and count vectorizer that naively maps the given query by tokenizing the words and compare with the database text. However, it is not an efficient approach both interms of computation and similarity.
1) The tf-idf based approach computes the similarity score for every vector of the text from the database which is slow and in-efficient.
2) Tokenization using nltk with stop words is helpful upto a point, but extracting similarity with just vectorized tokens is doesn't capture semantic meanings.
3) Also the cosine similarity is not a good metric to compare two vectors for semantic similarity because cosine similarity only gives the direction in which the vector is pointing not how the meaning is structured in the sentences. Different vectors pointing in same direction have higher similarity which might indicate high similarity score for two different sentences.


### References
1) The follwoing [Medium]('https://arupnanda.medium.com/lowdown-on-vector-databases-part-4-56840110babb') article has a decent explanation of vector databases and its implementation details.
2) Dataset [Link]('https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots/data')